# Collection person locations within frame and building a csv dataset

### Notebook 3 

In this notebook I will take the progress in using the model to identify pedestrians within frame and build a datat set of pedestrian location within a single frame.

By logging the coordinates within the following frame I'll attempt to link that coordinates that have moved very slightly will likely be from the pedestrian in the previous frame.

From this I will build a csv data set to use for future location predicition

In [ ]:
import os
import sys
import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt
import cv2
import json
import re

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.path.abspath("../")
print("Root Directory: ", ROOT_DIR)
sys.path.append(ROOT_DIR)

from mrcnn import utils
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log

from scripts import coco
from scripts import list_file_info as lfi
from scripts import video_to_frames as vtf
from scripts.centroid_tracker import CentroidTracker
from scripts.trackable_object import TrackableObject

In [ ]:
data_root = "/home/sam/Documents/GitHub/honoursProject/"

file_list = os.listdir(path = data_root) 
max = len(file_list) 

if max == 0:
    print("No video folders in directory")
else:
    print("please select video from 1 to " + str(max))

In [ ]:
# Select a video 

selected_video = 4

vid_dir = "/home/sam/Documents/GitHub/honoursProject/data/video/"
frames_dir = "/home/sam/Documents/GitHub/honoursProject/data/frames/"
final_vid_dir = "/home/sam/Documents/GitHub/honoursProject/data/output_vid/"
output_2_dir = "/home/sam/Documents/GitHub/honoursProject/data/output_image_2_frame/"
output_5_dir = "/home/sam/Documents/GitHub/honoursProject/data/output_image_5_frame/"
output_10_dir = "/home/sam/Documents/GitHub/honoursProject/data/output_image_10_frame/"
json_output_dir = "/home/sam/Documents/GitHub/honoursProject/data/output_json/"

In [ ]:
class InferenceConfig(coco.CocoConfig):
    # Set batch size to 1 since we'll be running inference on
    # one image at a time. Batch size = GPU_COUNT * IMAGES_PER_GPU
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

In [ ]:
# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "weights/mask_rcnn_coco.h5")
# Download COCO trained weights from Releases if needed
if not os.path.exists(COCO_MODEL_PATH):
    utils.download_trained_weights(COCO_MODEL_PATH)

class_names = ['BG', 'person', 'bicycle', 'car', 'motorcycle', 'airplane',
                'bus', 'train', 'truck', 'boat', 'traffic light',
                'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird',
                'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear',
                'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie',
                'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball',
                'kite', 'baseball bat', 'baseball glove', 'skateboard',
                'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup',
                'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
                'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza',
                'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed',
                'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote',
                'keyboard', 'cell phone', 'microwave', 'oven', 'toaster',
                'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
                'teddy bear', 'hair drier', 'toothbrush']

config = InferenceConfig()
config.display()

In [6]:
# Create model object in inference mode.
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)

print("Loading weights ", COCO_MODEL_PATH)

# Load weights trained on MS-COCO
model.load_weights(COCO_MODEL_PATH, by_name=True)

In [ ]:
# Get video file name 

vid_name = lfi.get_file_name(vid_dir)

video_path = vid_dir + vid_name
print(video_path)

In [ ]:
# Check if frames exist and if not break video into frames

frame_list = os.listdir(path = frames_dir) 
max = len(frame_list) 

if max == 0:
    vtf.getFrames(video_path, frames_dir)
    print("Video broken down to frames")
    frame_list = os.listdir(path = frames_dir)
    max = len(frame_list) 
else:
    print("frames already exist")

print("You can select from frame 0 to " + str(max - 1))

In [ ]:
# Create collection for frame names in frames_dir

frames_list = lfi.get_file_names(frames_dir)

# Sort the frames into accending order

frames_list.sort(key=lambda f: int(re.sub('\D', '', f)))

In [ ]:
# Create paths to frames 
frame_path_1 = os.path.join(frames_dir,frames_list[540])
frame_path_2 = os.path.join(frames_dir,frames_list[542])
frame_path_3 = os.path.join(frames_dir,frames_list[544])
frame_path_4 = os.path.join(frames_dir,frames_list[546])
frame_path_5 = os.path.join(frames_dir,frames_list[548])
frame_path_6 = os.path.join(frames_dir,frames_list[550])
frame_path_7 = os.path.join(frames_dir,frames_list[552])
frame_path_8 = os.path.join(frames_dir,frames_list[554])
frame_path_9 = os.path.join(frames_dir,frames_list[556])
frame_path_10 = os.path.join(frames_dir,frames_list[558])
frame_path_11 = os.path.join(frames_dir,frames_list[560])
frame_path_12 = os.path.join(frames_dir,frames_list[562])
frame_path_13 = os.path.join(frames_dir,frames_list[564])
frame_path_14 = os.path.join(frames_dir,frames_list[566])

# Create frames directory
frame_paths = [frame_path_1, frame_path_2, frame_path_3, frame_path_4, frame_path_5, 
                frame_path_6, frame_path_7, frame_path_8, frame_path_9, frame_path_10, 
                frame_path_11, frame_path_12, frame_path_13, frame_path_14]

# Create frames dictionary to hold each frame and the pedestrians within it
frames_dict = dict()

# Frame counter 
frame_count = 0

# Begin loop through frames in frame directory
for frame_path in frame_paths:
    
    # Set plot details for visulisation
    _, ax = plt.subplots(1, figsize=(10, 10))
    
    # Open cv analyses frame data
    frame = cv2.imread(frame_path)
    
    # Run object detection
    results = model.detect([frame], verbose=0)
    
    # collect results
    r = results[0]
    
    # Collect labels for display
    labels =[]
    
    # Create object dictionary to hold objects detected within results
    objects_dict = dict()
    
    # Objects counter
    objects_count = 0
    
    # Pedestrian counter
    pedestrian_count = 0
    
    # Loop through all identified class id's 
    for id in r['class_ids']:
        
        # If class id is equal to 1 its a person
        if id == 1:
            
            # Instantiate Centroid Tracker class
            ct = CentroidTracker()
            
            # Add person plus person count to labels for visulisation
            labels.append(str(pedestrian_count))
            
            # create collection for coordiunates for centroid calculation
            rects = []
            
            # Set Coordinates of pedestrian for centroid calculation
            # x1 = top
            top = r['rois'][objects_count][1]
            # y1 = left
            left = r['rois'][objects_count][0]
            # x2 = width
            width = r['rois'][objects_count][3]
            # y2 = height
            height = r['rois'][objects_count][2]
            
            # Add coordinates to rects 
            rects.append([left, top, left + width, top + height])
            
            # Get centroid results for pedestrian returns: (id, array([ x, y])) id not working so ignore
            centroid = ct.update(rects)
            
            # Get centroid X and Y
            X = centroid[0][0]
            Y = centroid[0][1]
            
            # Add person object to objects dictionary with centroid coordinates
            objects_dict[objects_count] = {'pedestrian ' + str(pedestrian_count): (X, Y)}
            
            # Add 1 to pedestrian total count
            pedestrian_count += 1
        
        # Else conditional for all other objects detected
        else:
            # Adds empty string to labels
            labels.append("")
            
            
        
        # Add 1 to objects total count
        objects_count += 1
    
    # Concatonate the frame number from frame path 
    frame_number = frame_path.replace("/home/sam/Documents/GitHub/honoursProject/4/frames/", "").replace(".jpg", "")
    
    # Add frame with frame number to frames dictionary with the objects dictionary nested
    frames_dict[frame_count] = {'frame ' + frame_number : objects_dict}

    image_ir = visualize.display_instances(frame, r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'], ax=ax,
                                title="Frame " + str(frame_number), captions=labels)
    
    plt.savefig(os.path.join(output_2_dir,frame_number), bbox_inches='tight')
    
    # Add 1 to frames total count
    frame_count += 1

# Loop through frames dictionary
for fd in frames_dict:
    # Print the frame data for each dictionary
    print(frames_dict[fd])

In [ ]:
# Create paths to frames 
frame_path_1 = os.path.join(frames_dir,frames_list[540])
frame_path_2 = os.path.join(frames_dir,frames_list[545])
frame_path_3 = os.path.join(frames_dir,frames_list[550])
frame_path_4 = os.path.join(frames_dir,frames_list[555])
frame_path_5 = os.path.join(frames_dir,frames_list[560])
frame_path_6 = os.path.join(frames_dir,frames_list[565])
frame_path_7 = os.path.join(frames_dir,frames_list[570])
frame_path_8 = os.path.join(frames_dir,frames_list[575])
frame_path_9 = os.path.join(frames_dir,frames_list[580])
frame_path_10 = os.path.join(frames_dir,frames_list[585])
frame_path_11 = os.path.join(frames_dir,frames_list[590])
frame_path_12 = os.path.join(frames_dir,frames_list[595])
frame_path_13 = os.path.join(frames_dir,frames_list[600])
frame_path_14 = os.path.join(frames_dir,frames_list[605])

# Create frames directory
frame_paths = [frame_path_1, frame_path_2, frame_path_3, frame_path_4, frame_path_5, 
                frame_path_6, frame_path_7, frame_path_8, frame_path_9, frame_path_10, 
                frame_path_11, frame_path_12, frame_path_13, frame_path_14]

# Create frames dictionary to hold each frame and the pedestrians within it
frames_dict = dict()

# Frame counter 
frame_count = 0

# Begin loop through frames in frame directory
for frame_path in frame_paths:
    
    # Set plot details for visulisation
    _, ax = plt.subplots(1, figsize=(10, 10))
    
    # Open cv analyses frame data
    frame = cv2.imread(frame_path)
    
    # Run object detection
    results = model.detect([frame], verbose=0)
    
    # collect results
    r = results[0]
    
    # Collect labels for display
    labels =[]
    
    # Create object dictionary to hold objects detected within results
    objects_dict = dict()
    
    # Objects counter
    objects_count = 0
    
    # Pedestrian counter
    pedestrian_count = 0
    
    # Loop through all identified class id's 
    for id in r['class_ids']:
        
        # If class id is equal to 1 its a person
        if id == 1:
            
            # Instantiate Centroid Tracker class
            ct = CentroidTracker()
            
            # Add person plus person count to labels for visulisation
            labels.append(str(pedestrian_count))
            
            # create collection for coordiunates for centroid calculation
            rects = []
            
            # Set Coordinates of pedestrian for centroid calculation
            # x1 = top
            top = r['rois'][objects_count][1]
            # y1 = left
            left = r['rois'][objects_count][0]
            # x2 = width
            width = r['rois'][objects_count][3]
            # y2 = height
            height = r['rois'][objects_count][2]
            
            # Add coordinates to rects 
            rects.append([left, top, left + width, top + height])
            
            # Get centroid results for pedestrian returns: (id, array([ x, y])) id not working so ignore
            centroid = ct.update(rects)
            
            # Get centroid X and Y
            X = centroid[0][0]
            Y = centroid[0][1]
            
            # Add person object to objects dictionary with centroid coordinates
            objects_dict[objects_count] = {'pedestrian ' + str(pedestrian_count): (X, Y)}
            
            # Add 1 to pedestrian total count
            pedestrian_count += 1
        
        # Else conditional for all other objects detected
        else:
            # Adds empty string to labels
            labels.append("")
            
            
        
        # Add 1 to objects total count
        objects_count += 1
    
    # Concatonate the frame number from frame path 
    frame_number = frame_path.replace("/home/sam/Documents/GitHub/honoursProject/4/frames/", "").replace(".jpg", "")
    
    # Add frame with frame number to frames dictionary with the objects dictionary nested
    frames_dict[frame_count] = {'frame ' + frame_number : objects_dict}

    image_ir = visualize.display_instances(frame, r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'], ax=ax,
                                title="Frame " + str(frame_number), captions=labels)
    
    plt.savefig(os.path.join(output_5_dir,frame_number), bbox_inches='tight')
    
    # Add 1 to frames total count
    frame_count += 1

# Loop through frames dictionary
for fd in frames_dict:
    # Print the frame data for each dictionary
    print(frames_dict[fd])

In [ ]:
# Create paths to frames 
frame_path_1 = os.path.join(frames_dir,frames_list[520])
frame_path_2 = os.path.join(frames_dir,frames_list[530])
frame_path_3 = os.path.join(frames_dir,frames_list[540])
frame_path_4 = os.path.join(frames_dir,frames_list[550])
frame_path_5 = os.path.join(frames_dir,frames_list[560])
frame_path_6 = os.path.join(frames_dir,frames_list[570])
frame_path_7 = os.path.join(frames_dir,frames_list[580])
frame_path_8 = os.path.join(frames_dir,frames_list[590])
frame_path_9 = os.path.join(frames_dir,frames_list[600])
frame_path_10 = os.path.join(frames_dir,frames_list[610])
frame_path_11 = os.path.join(frames_dir,frames_list[620])
frame_path_12 = os.path.join(frames_dir,frames_list[630])
frame_path_13 = os.path.join(frames_dir,frames_list[640])
frame_path_14 = os.path.join(frames_dir,frames_list[650])

# Create frames directory
frame_paths = [frame_path_1, frame_path_2, frame_path_3, frame_path_4, frame_path_5, 
                frame_path_6, frame_path_7, frame_path_8, frame_path_9, frame_path_10, 
                frame_path_11, frame_path_12, frame_path_13, frame_path_14]

# Create frames dictionary to hold each frame and the pedestrians within it
frames_dict = dict()

# Frame counter 
frame_count = 0

# Begin loop through frames in frame directory
for frame_path in frame_paths:
    
    # Set plot details for visulisation
    _, ax = plt.subplots(1, figsize=(10, 10))
    
    # Open cv analyses frame data
    frame = cv2.imread(frame_path)
    
    # Run object detection
    results = model.detect([frame], verbose=0)
    
    # collect results
    r = results[0]
    
    # Collect labels for display
    labels =[]
    
    # Create object dictionary to hold objects detected within results
    objects_dict = dict()
    
    # Objects counter
    objects_count = 0
    
    # Pedestrian counter
    pedestrian_count = 0
    
    # Loop through all identified class id's 
    for id in r['class_ids']:
        
        # If class id is equal to 1 its a person
        if id == 1:
            
            # Instantiate Centroid Tracker class
            ct = CentroidTracker()
            
            # Add person plus person count to labels for visulisation
            labels.append(str(pedestrian_count))
            
            # create collection for coordiunates for centroid calculation
            rects = []
            
            # Set Coordinates of pedestrian for centroid calculation
            # x1 = top
            top = r['rois'][objects_count][1]
            # y1 = left
            left = r['rois'][objects_count][0]
            # x2 = width
            width = r['rois'][objects_count][3]
            # y2 = height
            height = r['rois'][objects_count][2]
            
            # Add coordinates to rects 
            rects.append([left, top, left + width, top + height])
            
            # Get centroid results for pedestrian returns: (id, array([ x, y])) id not working so ignore
            centroid = ct.update(rects)
            
            # Get centroid X and Y
            X = centroid[0][0]
            Y = centroid[0][1]
            
            # Add person object to objects dictionary with centroid coordinates
            objects_dict[objects_count] = {'pedestrian ' + str(pedestrian_count): (X, Y)}
            
            # Add 1 to pedestrian total count
            pedestrian_count += 1
        
        # Else conditional for all other objects detected
        else:
            # Adds empty string to labels
            labels.append("")
            
            
        
        # Add 1 to objects total count
        objects_count += 1
    
    # Concatonate the frame number from frame path 
    frame_number = frame_path.replace("/home/sam/Documents/GitHub/honoursProject/4/frames/", "").replace(".jpg", "")
    
    # Add frame with frame number to frames dictionary with the objects dictionary nested
    frames_dict[frame_count] = {'frame ' + frame_number : objects_dict}

    image_ir = visualize.display_instances(frame, r['rois'], r['masks'], r['class_ids'], 
                                class_names, r['scores'], ax=ax,
                                title="Frame " + str(frame_number), captions=labels)
    
    plt.savefig(os.path.join(output_10_dir,frame_number), bbox_inches='tight')
    
    # Add 1 to frames total count
    frame_count += 1

# Loop through frames dictionary
for fd in frames_dict:
    # Print the frame data for each dictionary
    print(frames_dict[fd])

In [ ]:
# Making dataset from the 3 pedestrians movments 

# Pedestrian 0
person_0_X = [767, 752, 746, 716, 717, 711, 699, 690, 689, 689]

person_0_Y = [1243, 1217, 1188, 1148, 1126, 1101, 1078, 1073, 1065, 1058]

# Pedestrian 1
person_1_X = [803, 776, 767, 759, 742, 741, 736, 722, 719, 714]

person_1_Y = [1368, 1324, 1295, 1266, 1235, 1217, 1191, 1171, 1154, 1138]

# Pedestrian 2
person_2_X = [782, 774, 789, 766, 760, 753, 739, 739, 725, 729]

person_2_Y = [1452, 1422, 1367, 1332, 1305, 1271, 1246, 1228, 1212, 1186]

### What should happen next

The above would be repeated for each persons X and Y coordinates giving a prediciton for there positoions 10 frames in future

The frame would be anaylsed by MRCNN

Bounding box centroid compared to memspectrum average 

Have picture with dot from guess and dot from bounding box 

#### outputted results

Pedestrians in the outputted frames are in the same order in each picture but the identification will identify them in a different order

person original order is 0 1 2 I will put the correct order under each output:


{'frame 540': {0: {'pedestrian 0': (767, 1243)}, 1: {'pedestrian 1': (803, 1368)}, 3: {'pedestrian 2': (782, 1452)}}}

0 1 2

{'frame 550': {0: {'pedestrian 0': (752, 1217)}, 1: {'pedestrian 1': (776, 1324)}, 2: {'pedestrian 2': (774, 1422)}}}

0 1 2

{'frame 560': {0: {'pedestrian 0': (789, 1367)}, 1: {'pedestrian 1': (746, 1188)}, 2: {'pedestrian 2': (767, 1295)}}}

1 2 0

{'frame 570': {0: {'pedestrian 0': (759, 1266)}, 1: {'pedestrian 1': (766, 1332)}, 2: {'pedestrian 2': (716, 1148)}}}

2 0 1

{'frame 580': {0: {'pedestrian 0': (742, 1235)}, 1: {'pedestrian 1': (717, 1126)}, 2: {'pedestrian 2': (760, 1305)}}}

1 0 2

{'frame 590': {0: {'pedestrian 0': (741, 1217)}, 1: {'pedestrian 1': (753, 1271)}, 2: {'pedestrian 2': (711, 1101)}}}

2 0 1

{'frame 600': {0: {'pedestrian 0': (736, 1191)}, 1: {'pedestrian 1': (739, 1246)}, 2: {'pedestrian 2': (699, 1078)}}}

2 0 1

{'frame 610': {0: {'pedestrian 0': (739, 1228)}, 1: {'pedestrian 1': (722, 1171)}, 2: {'pedestrian 2': (690, 1073)}}}

2 1 0

{'frame 620': {0: {'pedestrian 0': (725, 1212)}, 1: {'pedestrian 1': (719, 1154)}, 2: {'pedestrian 2': (689, 1065)}}}

2 1 0

{'frame 630': {0: {'pedestrian 0': (714, 1138)}, 1: {'pedestrian 1': (729, 1186)}, 2: {'pedestrian 2': (689, 1058)}}}

2 0 1